## Model training on AWS Sagemaker

**Author:** Shaun Khoo  
**Date:** 15 Oct 2021  
**Context:** Training on the local computer is taking too much time, would be much faster if we could train our models on AWS Sagemaker instead  
**Objective:** Develop code that will help us train our model directly on AWS Sagemaker   

**Note:** Referencing [this notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/pytorch_lstm_word_language_model/pytorch_rnn.ipynb)

#### A) Importing the required libraries

Note that your AWS credentials need to be set up on the AWS CLI first before this can work seamlessly

In [4]:
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

The code below returns your IAM user (as an Amazon Resource Number or `Arn`). Make sure the code runs below - this ensures you are logged in correctly

In [5]:
sts = boto3.client('sts')
sts.get_caller_identity()

{'UserId': 'AIDAYUZMQUYGUNJ2VXD2E',
 'Account': '594409465357',
 'Arn': 'arn:aws:iam::594409465357:user/shaunkhoo',
 'ResponseMetadata': {'RequestId': '314e2f55-26ea-4a35-8e43-b1d9e587238b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '314e2f55-26ea-4a35-8e43-b1d9e587238b',
   'content-type': 'text/xml',
   'content-length': '406',
   'date': 'Wed, 29 Dec 2021 06:31:34 GMT'},
  'RetryAttempts': 0}}

Changing the working directory to the top-level folder

In [6]:
import os
os.chdir('..')

#### B) Setting up Sagemaker and S3

Initialising the Sagemaker session object

In [7]:
sagemaker_session = sagemaker.Session()

Obtaining the default bucket for our Sagemaker session

In [8]:
bucket = sagemaker_session.default_bucket()
print(f"Bucket Name: {bucket}")

Bucket Name: sagemaker-us-east-1-594409465357


Set the prefix for where you want to store your data / model files in the S3 bucket

In [9]:
prefix = 'Sagemaker/ssoc-autocoder'

Run the code below to retrieve Sagemaker's execution role. Note that the role we have set up is called `mom-aws`

In [10]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName = 'mom-aws')['Role']['Arn']
print(f"Execution Role: {role}")

Couldn't call 'get_role' to get Role ARN from role name shaunkhoo to get Role path.


Execution Role: arn:aws:iam::594409465357:role/mom-aws


Upload the raw data to the S3 folder

In [13]:
inputs = sagemaker_session.upload_data(path = "Data/Train/pre-training-sample1000.txt", 
                                       bucket = bucket, 
                                       key_prefix = prefix)

In [14]:
print(f"Inputs stored in: {inputs}")

Inputs stored in: s3://sagemaker-us-east-1-594409465357/Sagemaker/ssoc-autocoder/pre-training-sample1000.txt


#### C) Language modelling (or pretraining) on Sagemaker

Running masked language modelling to finetune the DistilBERT model on MCF data to improve downstream classification performance

Define the hyperparameters that need to be passed onto the masked language modelling script

In [11]:
mlm_parameters = {
    'model_name_or_path': 'distilbert-base-uncased',
    'train_file': "pre-training-full.txt",
    'line_by_line': True,
    'do_train': True,
    'do_eval': True,
    'evaluation_strategy': 'epoch',
    'logging_steps': 500,
    'save_strategy': 'no',
    'overwrite_output_dir': True,
    'output_dir': '20211228_test'
}

Create the estimator object and run it on the full pretraining text file

In [12]:
mlm_estimator = PyTorch(
    entry_point = "run_mlm_aws.py",
    role = role,
    framework_version = "1.8.1",
    instance_count = 1,
    instance_type = "ml.g4dn.xlarge",
    source_dir = "ssoc_autocoder",
    py_version = "py3",
    env = {'SAGEMAKER_REQUIREMENTS': '../Notebooks/aws-training-requirements/pretrain/requirements.txt'},
    hyperparameters = mlm_parameters,
)

In [ ]:
mlm_estimator.fit({"training": 's3://sagemaker-us-east-1-594409465357/Sagemaker/ssoc-autocoder/pre-training-full.txt'})

2021-12-29 06:32:02 Starting - Starting the training job...
2021-12-29 06:32:05 Starting - Launching requested ML instancesProfilerReport-1640759519: InProgress
...
2021-12-29 06:33:05 Starting - Preparing the instances for training......
2021-12-29 06:34:20 Downloading - Downloading input data......
2021-12-29 06:35:24 Training - Downloading the training image..................
2021-12-29 06:38:51 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-29 06:38:51,732 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-29 06:38:51,755 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-29 06:38:54,780 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-29 06:38:55,150 sagemaker-training-toolkit INFO     Installing dependencies from r

In [56]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point = "train_aws.py",
    role = role,
    framework_version = "1.8.1",
    instance_count = 1,
    instance_type = "ml.g4dn.xlarge",
    source_dir = "ssoc_autocoder",
    py_version = "py3",
    env = env,
    #use_spot_instances = True,
    #max_run = ,
    #max_wait = 600,
    hyperparameters = {"epochs": 1, "tied": True},
)

In [53]:
estimator.fit({"training": inputs})

2021-10-11 07:40:29 Starting - Starting the training job...
2021-10-11 07:40:55 Starting - Launching requested ML instancesProfilerReport-1633938053: InProgress
......
2021-10-11 07:42:04 Starting - Preparing the instances for training.........
2021-10-11 07:43:40 Downloading - Downloading input data...
2021-10-11 07:44:16 Training - Downloading the training image......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-11 07:48:29,655 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-11 07:48:29,679 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-11 07:48:29,689 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-11 07:48:30,459 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

202

KeyboardInterrupt: 

In [ ]:
estimator.fit({"training": inputs})

2021-10-11 08:58:49 Starting - Starting the training job...
2021-10-11 08:59:13 Starting - Launching requested ML instancesProfilerReport-1633942752: InProgress
...
2021-10-11 08:59:53 Starting - Preparing the instances for training.........
2021-10-11 09:01:37 Downloading - Downloading input data
2021-10-11 09:01:37 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-11 09:05:49,253 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-11 09:05:49,273 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-11 09:05:50,703 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-11 09:05:51,289 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2021-10-